# Импорт библиотек

In [1]:
import os
from os.path import exists, join, basename, splitext
from pathlib import Path
import cv2
import math 
import pandas as pd
import json  
from pprint import pprint  
import numpy as np
from scipy import stats
from sklearn import preprocessing
from google.colab.patches import cv2_imshow

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Загрузка модели


In [3]:
git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

Selecting previously unselected package libgflags2.2.
(Reading database ... 124013 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

# Запуск модели на пользовательских данных

In [4]:
!cd openpose && ./build/examples/openpose/openpose.bin \
--video /content/drive/MyDrive/SF/Project_5/data/student.mp4 \
--keypoint_scale 1 \
--net_resolution '656x368' \
--write_json ./output/ \
--display 0 \
--write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output_student.mp4

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
Empty frame detected, frame number 3697 of 3700. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():290
Empty frame detected, frame number 3697 of 3700. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():290
Empty frame detected, frame number 3697 of 3700. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():290
OpenPose demo successfully finished. Total time: 694.521345 seconds.
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enabl

In [5]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

In [6]:
show_local_mp4_video('output_student.mp4')

Output hidden; open in https://colab.research.google.com to view.

In [7]:
!cd openpose && ./build/examples/openpose/openpose.bin \
--video /content/drive/MyDrive/SF/Project_5/data/coach.mp4 \
--keypoint_scale 1 \
--net_resolution '656x368' \
--display 0  \
--write_video ../openpose.avi \
--write_json ./output/ \
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output_coach.mp4

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
Empty frame detected, frame number 8937 of 8940. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():290
Empty frame detected, frame number 8937 of 8940. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():290
Empty frame detected, frame number 8937 of 8940. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():290
OpenPose demo successfully finished. Total time: 1693.666745 seconds.
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enab

In [8]:
show_local_mp4_video('output_coach.mp4')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#!cp -a /content/openpose/output/* /content/drive/MyDrive/SF/Project_5/k_points/

# Чтение полученных файлов и сохранение всех опорных точек

In [10]:
# путь с хранением файлов .json
path_folder = '/content/openpose/output/'

In [11]:
cap_st = cv2.VideoCapture('output_student.mp4')
frameRate_st = cap_st.get(5)  # частота кадров
prop_fps_st = cap_st.get(7) # количество фреймов

cap_ch = cv2.VideoCapture('output_coach.mp4')
frameRate_ch = cap_ch.get(5)  # частота кадров
prop_fps_ch = cap_ch.get(7) # количество фреймов

In [12]:
dict_video = {'id': [0,1],
              'role': ['student', 'coach'],
              'frame_frequency': [int(frameRate_st),int(frameRate_ch)],
              'num_frame': [int(prop_fps_st),int(prop_fps_ch)]}

In [13]:
def get_kpoints(file):
  k_points_count = []
  
  # проход по всем найденным людям
  for person in file['people']:
    # подсчет обнаруженных опорных точек у каждой персоны
    # деление на 3 - по каждой точке данны 2 кординаты и 1 вероятность
    non_zero = len([item for item in person['pose_keypoints_2d'] if item != 0])/3
    k_points_count.append(non_zero)
  
  # номер человека с максимальным количеством детектированных точек
  id_person = k_points_count.index(max(k_points_count))
  # ключевые точки выбранной персоны
  k_points =file['people'][id_person]['pose_keypoints_2d']
  # перевод данных в numpy размером 25*3 (25-количество точек. идентифицируемых моделью)
  k_points_person = np.array(k_points).reshape((25, 3))
  
  return k_points_person

In [14]:
data_st = []
data_ch = []
for index_role in dict_video.get('id'):
  # проход по всем файлам с соответствующей частотой
    for index_file in range (0,
                             dict_video.get('num_frame')[index_role],
                             dict_video.get('frame_frequency')[index_role]):
      path_file = path_folder + dict_video.get('role')[index_role] + '_' +  ('%012d'% (index_file)) + '_keypoints.json'
      
      # запись всех опорных точек в соответствующие переменные
      with open(path_file) as f:
        file = json.load(f)
      if index_role == 0:
        data_st.append(get_kpoints(file))
      else:
        data_ch.append(get_kpoints(file)) 

# Редактирование данных для расчета метрик

In [15]:
num_frame = len(data_st)

In [16]:
coach_key_points = []
student_key_points = []
for frame in range(num_frame):
  # поиск нулевых векторов на фрейме студента
  id_zero_st = np.where(data_st[frame].sum(axis=1) == 0)
  # поиск нулевых векторов на фрейме коуча
  id_zero_ch = np.where(data_ch[frame].sum(axis=1) == 0)
  # объединение индексов нулевых векторов
  id_pass = np.concatenate((id_zero_st[0],id_zero_ch[0])).astype(int)
  # удаление опорных точек, нулевых либо у тренера, либо у студента
  student_key_points.append(np.delete(data_st[frame], np.unique(id_pass), axis = 0))
  coach_key_points.append(np.delete(data_ch[frame], np.unique(id_pass), axis = 0))

# Расчёт метрик

In [17]:
def cos_similarity(pose1, pose2):
    # косинусное сходство между всеми ключевыми точками размерностью = 25*25
    cossim = pose1.dot(np.transpose(pose2)) / (
        np.linalg.norm(pose1, axis=1) * np.linalg.norm(pose2, axis=1)
    )
    # косинусное сходство между сооответствующими ключевыми точками
    cossim_pair = np.diagonal(cossim)

    # усредненное косинусное сходство по фигуре
    cossim_person = cossim_pair.mean()

    return cossim_person

In [18]:
def weight_distance(pose1, pose2, conf1):
    # D(U,V) = (1 / sum(conf1)) * sum(conf1 * ||pose1 - pose2||) = sum1 * sum2

    sum1 = 1 / np.sum(conf1)
    sum2 = 0

    for i in range(len(pose1)):
        # каждый индекс i имеет x и y, у которых одинаковая оценка достоверности
        sum2 += np.sum(conf1[i] * abs(pose1[i] - pose2[i]))

    weighted_dist = sum1 * sum2

    return weighted_dist

In [19]:
# создание набора метрик
cos_sim = []
distance = []

# создание набора метрик для вывода на печать
cos_sim_print = []
distance_print = []

for frame in range(num_frame):
  cos_sim_pose = cos_similarity(student_key_points[frame][:,:2],
                                coach_key_points[frame][:,:2])
  distance_pose = weight_distance(student_key_points[frame][:,:2],
                                  coach_key_points[frame][:,:2],
                                  student_key_points[frame][:,-1])
  
  cos_sim_pose_print = str(f"{cos_sim_pose:.0%}")
  distance_pose_print = '%d'% (distance_pose)

  cos_sim.append(cos_sim_pose)
  distance.append(distance_pose)

  cos_sim_print.append(cos_sim_pose_print)
  distance_print.append(distance_pose_print)

# Нанесение метрик на видео

In [20]:
step_save = 6
folder_person = 'person_skeleton'
if not os.path.exists(folder_person):
  os.makedirs(folder_person)

In [21]:
cap = cv2.VideoCapture('output_student.mp4')
frameRate = cap.get(5)  # частота кадров

# Определение кодека и создание видеозаписи
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)

fourcc = cv2.VideoWriter_fourcc(*'MPEG')
out = cv2.VideoWriter('output_video.mp4', fourcc, 2.0, size)

id_metric = 0
while (cap.isOpened()):
    frameId = cap.get(1) # номер текущего кадра
    ret, frame = cap.read() 
    if (ret != True):
      break
    elif (frameId % math.floor(frameRate) == 0):
      font = cv2.FONT_HERSHEY_SIMPLEX

      cv2.putText(frame,
                  ''.join(['Limb direction ', cos_sim_print[id_metric]]),
                  (50, 50), 
                  font, 0.5, 
                  (255, 0, 0), 
                  1, 
                  cv2.LINE_4
                  )
      
      cv2.putText(frame,
                  ''.join([' Pose distance ', distance_print[id_metric]]),
                  (50, 75), 
                  font, 0.5, 
                  (255, 0, 0), 
                  1, 
                  cv2.LINE_4
                  )
      # сохранение одного фрейма из секундного интервала в формате изображения
      if (frameId % frameRate) % step_save == 0:
        filename ="frame%d.jpg" % (frameId/frameRate)
        directory = os.path.join(folder_person, filename)
        cv2.imwrite(directory, frame)
      # выход из фрейма
      out.write(frame)
      id_metric+=1
        
      # ключ для выхода из записи
      if cv2.waitKey(1) & 0xFF == ord('a'):
          break
  
cap.release()
out.release()
cv2.destroyAllWindows()

# Сохранение паралельных кадров

In [22]:
cap = cv2.VideoCapture('output_coach.mp4')
frameRate = cap.get(5)  # частота кадров

# создание папки для вывода 
folder_tandem = 'tandem'
if not os.path.exists(folder_tandem):
  os.makedirs(folder_tandem)

while (cap.isOpened()):
    frameId = cap.get(1) # номер текущего кадра
    ret, frame = cap.read() 
    if (ret != True):
      break
      
    # обработка фрейма из каждого шестого секундного интервала
    elif (frameId % (frameRate*step_save)) == 0:

      # изменение размера изображения коуча
      resized_coach = cv2.resize(frame, size,
                                 interpolation = cv2.INTER_AREA)
      
      # открытие изображения со студентом на той же секунде
      num_second = int(frameId/frameRate)
      path = os.path.join(folder_person,'frame'+str(num_second)+'.jpg')
      img_person = cv2.imread(path) 
      
      # сохранение файла c коучем и студентом
      filename ="coach_person_%d.jpg" % num_second
      directory = os.path.join(folder_tandem, filename)
      tandem_img = np.concatenate((resized_coach, img_person),
                                  axis=1)
      cv2.imwrite(directory, tandem_img)
      
      # выход из фрейма
      out.write(frame)

      # ключ для выхода из записи
      if cv2.waitKey(1) & 0xFF == ord('a'):
          break
  
cap.release()
out.release()
cv2.destroyAllWindows()

In [35]:
# количество ключевых точек на всех фотографиях
num_kpoints_person = 25*num_frame
# количество учтённых ключевых точек
num_kpoints = 0
for frame in range(num_frame):
  num_kpoints += student_key_points[frame].shape[0]
print(f'Average limb direction = {(stats.mode(cos_sim)[0].item()):.2%}')
print(f"Average pose distance = {'%d'%(stats.mode(distance)[0].item())}")
print('0% images were unvalued')
print(f'{(1- num_kpoints/num_kpoints_person):.0%} key points were deleted')

Average limb direction = 98.32%
Average pose distance = 49
0% images were unvalued
21% key points were deleted


In [24]:
 !zip -r person.zip person_skeleton/

  adding: person_skeleton/ (stored 0%)
  adding: person_skeleton/frame32.jpg (deflated 0%)
  adding: person_skeleton/frame8.jpg (deflated 0%)
  adding: person_skeleton/frame74.jpg (deflated 0%)
  adding: person_skeleton/frame26.jpg (deflated 0%)
  adding: person_skeleton/frame41.jpg (deflated 0%)
  adding: person_skeleton/frame138.jpg (deflated 0%)
  adding: person_skeleton/frame59.jpg (deflated 0%)
  adding: person_skeleton/frame113.jpg (deflated 0%)
  adding: person_skeleton/frame79.jpg (deflated 0%)
  adding: person_skeleton/frame139.jpg (deflated 0%)
  adding: person_skeleton/frame9.jpg (deflated 0%)
  adding: person_skeleton/frame129.jpg (deflated 0%)
  adding: person_skeleton/frame0.jpg (deflated 0%)
  adding: person_skeleton/frame135.jpg (deflated 0%)
  adding: person_skeleton/frame73.jpg (deflated 0%)
  adding: person_skeleton/frame101.jpg (deflated 0%)
  adding: person_skeleton/frame131.jpg (deflated 0%)
  adding: person_skeleton/frame37.jpg (deflated 0%)
  adding: person_skel

In [25]:
 !zip -r tandem.zip tandem/

  adding: tandem/ (stored 0%)
  adding: tandem/coach_person_102.jpg (deflated 0%)
  adding: tandem/coach_person_96.jpg (deflated 0%)
  adding: tandem/coach_person_24.jpg (deflated 0%)
  adding: tandem/coach_person_78.jpg (deflated 0%)
  adding: tandem/coach_person_84.jpg (deflated 0%)
  adding: tandem/coach_person_126.jpg (deflated 0%)
  adding: tandem/coach_person_18.jpg (deflated 0%)
  adding: tandem/coach_person_36.jpg (deflated 0%)
  adding: tandem/coach_person_66.jpg (deflated 0%)
  adding: tandem/coach_person_12.jpg (deflated 0%)
  adding: tandem/coach_person_60.jpg (deflated 0%)
  adding: tandem/coach_person_144.jpg (deflated 0%)
  adding: tandem/coach_person_72.jpg (deflated 0%)
  adding: tandem/coach_person_132.jpg (deflated 0%)
  adding: tandem/coach_person_108.jpg (deflated 0%)
  adding: tandem/coach_person_42.jpg (deflated 0%)
  adding: tandem/coach_person_120.jpg (deflated 1%)
  adding: tandem/coach_person_0.jpg (deflated 1%)
  adding: tandem/coach_person_48.jpg (deflated 